In [44]:
import imaplib, email, os, json
from email.header import decode_header

from dotenv import load_dotenv
from langchain import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
load_dotenv()

True

login

In [45]:
IMAP_SERVER = 'imap.gmx.com'
IMAP_PORT = 993
EMAIL_ACCOUNT = os.environ['EMAIL']
EMAIL_PASSWORD = os.environ['EMAIL_PASSWORD']
mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
mail.login(EMAIL_ACCOUNT, EMAIL_PASSWORD)

('OK', [b'LOGIN completed'])

In [55]:
mail.select('INBOX')
status, messages = mail.search(None, 'ALL')
email_ids = messages[0].split()
email_ids = [int(eid) for eid in email_ids]

In [38]:
def get_email_body(msg):
    body = ""
    if msg.is_multipart():
        # If multipart, iterate through the parts
        for part in msg.walk():
            # Check if the content type is text/plain or text/html
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition"))

            if content_type == "text/plain" and "attachment" not in content_disposition:
                # Extract plain text
                body = part.get_payload(decode=True).decode(part.get_content_charset() or "utf-8")
                break
            elif content_type == "text/html" and "attachment" not in content_disposition:
                # Extract HTML
                body = part.get_payload(decode=True).decode(part.get_content_charset() or "utf-8")
    else:
        # If not multipart, check the content type
        content_type = msg.get_content_type()
        if content_type == "text/plain" or content_type == "text/html":
            body = msg.get_payload(decode=True).decode(msg.get_content_charset() or "utf-8")
    return body

In [39]:
llm = ChatOpenAI(model = 'gpt-4-turbo', temperature=0)

classification_prompt = PromptTemplate.from_template(
    '''
        Task:
        You are my (florian kenner's) personal assistant who is instructed to cleanup his mailbox. Rate the following email, whether to keep or delete it. You want to generally delete all emails which will not be of any importance in the future, such as ads, spam, newsletters outdated information, such as one time authentication codes etc.
        Keep all emails which are personal communications, bookings, invoices, important notifications etc.
        If you are not totally sure, keep the email.

        Email Title: {header}
        Email Sender: {sender}
        Date: {date}
        Email Content: {content}


           Format of Expected Output: Please provide your classification and justification in the following structured format:
            {{
                "classification": "KEEP" or "DELETE",
                "justification": "A one short sentence explanation of why you made this classification based on the provided email"
            }}
    '''
)

In [64]:
BACKUP_FOLDER = "./email_backups"

def save_email_backup(email_id, message, metadata):
    """
    Save email as a raw .eml file and metadata as a .json file.
    """
    # Create backup folder if not exists
    os.makedirs(BACKUP_FOLDER, exist_ok=True)

    # File names (sanitize subject to avoid invalid characters in file name)
    subject = metadata["subject"][:50].replace("/", "_").replace("\\", "_")
    eml_file = os.path.join(BACKUP_FOLDER, f"{email_id}_{subject}.eml")
    json_file = os.path.join(BACKUP_FOLDER, f"{email_id}_{subject}.json")

    # Save raw email content as .eml
    with open(eml_file, "wb") as f:
        f.write(message.as_bytes())

    # Save metadata as .json
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=4, ensure_ascii=False)

classify and delete

In [66]:
for id in email_ids[:1]:
    _, msg_data = mail.fetch(str(id), '(RFC822)')
    message = email.message_from_bytes(msg_data[0][1])
    subject, encoding = decode_header(message['subject'])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding if encoding else "utf-8")

    sender = message.get('FROM')
    date = message.get('Date')
    body = get_email_body(message)

    

    # Save the email to the folder email_backup
    metadata = {
    "email_id": id,
    "subject": subject,
    "sender": sender,
    "date": date,
    "body": body
}
    save_email_backup(id, message, metadata)


    agent = LLMChain(llm = llm, prompt = classification_prompt)

    result_raw = agent.invoke({
        'header': subject,
        'sender': sender,
        'date': date,
        'content': body
    })
    result_str = result_raw.get('text', '')
    result = json.loads(result_str)
    print(result['classification'])

KEEP
